In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
import gmaps 
import gmaps.datasets
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import Perceptron
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from scipy.stats import boxcox
from scipy.spatial import KDTree
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999
sns.set_theme()

In [2]:
#We import the custom functions that we will be using: import_dataset, rf_model, rf_clustering, evaluate
from helper_functions import import_dataset, rf_model, rf_clustering, evaluate


In [3]:
dataset = pd.read_excel(r"..\Data_Midterm.xls")


In [4]:
def clean_data(df):
    df = df.drop(columns=['id', "date", 'sqft_above'])
    df = df[df["bedrooms"] <= 6]
    df = df[(df["bathrooms"] < 6) & (df["bathrooms"] >= 1)]
    df = df[df["sqft_living"] <= 5000]
    df = df[df["sqft_living15"] <= 5000]
    df = df[df["sqft_lot"] <= 15000]
    df = df[df["sqft_lot15"] <= 15000]
    df.loc[df["view"] != 0, "view"] = 1
    df = df[df["condition"] > 2]
    df.loc[df["sqft_basement"] != 0, "sqft_basement"] = 1
    df.loc[(df["yr_renovated"]!=0)&(df["yr_renovated"]<1990), "yr_renovated"] = 1
    df.loc[df["yr_renovated"]>=1990, "yr_renovated"] = 2
    
  
    return df

In [5]:
clean_dataset = clean_data(dataset)
clean_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17821 entries, 0 to 21595
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       17821 non-null  int64  
 1   bathrooms      17821 non-null  float64
 2   sqft_living    17821 non-null  int64  
 3   sqft_lot       17821 non-null  int64  
 4   floors         17821 non-null  float64
 5   waterfront     17821 non-null  int64  
 6   view           17821 non-null  int64  
 7   condition      17821 non-null  int64  
 8   grade          17821 non-null  int64  
 9   sqft_basement  17821 non-null  int64  
 10  yr_built       17821 non-null  int64  
 11  yr_renovated   17821 non-null  int64  
 12  zipcode        17821 non-null  int64  
 13  lat            17821 non-null  float64
 14  long           17821 non-null  float64
 15  sqft_living15  17821 non-null  int64  
 16  sqft_lot15     17821 non-null  int64  
 17  price          17821 non-null  int64  
dtypes: flo

In [6]:
X_train, X_test, y_train, y_test = train_test_split(clean_dataset.drop(["price", "yr_built"],axis=1), clean_dataset["price"])

scaler = StandardScaler()
reg = LinearRegression()
pipe = make_pipeline(scaler, reg)
pipe.fit(X_train, y_train)

y_predict = pipe.predict(X_test)

results = evaluate(pipe, X_test, y_test, silent=False)

---Model Performance---
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

Average Absolute Error: 109609.6 dollars.
Mean squared error: 162025.97
Accuracy = 76.48%.
Coefficient of determination: 0.67



In [7]:
reg_clustering, centers_df, X_train, y_train, X_test, y_test = rf_clustering(clean_dataset)
results = evaluate(reg_clustering, X_test, y_test, silent=False)

c:\users\carlo\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


---Model Performance---
RandomForestRegressor(bootstrap=False, max_depth=100, max_features='sqrt',
                      n_estimators=300)

Average Absolute Error: 56661.0 dollars.
Mean squared error: 97008.04
Accuracy = 88.01%.
Coefficient of determination: 0.88

